<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 12: Ghostbusters, Inc., exorcises (gets rid of) ghosts.
During each of the next three months, the company will
receive the following number of calls from people who want
their ghosts exorcised: January, 100 calls; February, 300
calls; March, 200 calls. Ghostbusters is paid \$800 for each
ghost exorcised during the month in which the customer
calls. Calls need not be responded to during the month they
are made, but if a call is responded to one month after it is
made, then Ghostbusters loses $100 in future goodwill, and
if a call is responded to two months after it is made,
Ghostbusters loses \$200 in goodwill. Each employee of
Ghostbusters can exorcise 10 ghosts during a month. Each
employee is paid a salary of \$4,000 per month. At the
beginning of January, the company has 8 workers. Workers
can be hired and trained (in 0 time) at a cost of \$5,000 per
worker. Workers can be fired at a cost of \$4,000 per worker.
Formulate an LP to maximize Ghostbusters’ profit (revenue
less costs) over the next three months. Assume that all calls
must be handled by the end of March.

In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.2 MB/s eta 0:00:00


In [3]:
from pulp import LpProblem, LpMaximize, LpVariable, LpInteger, lpSum, value, LpStatus

# --- Data --- #
months = [1,2,3]            # 1=Jan, 2=Feb, 3=Mar
D = {1:100, 2:300, 3:200}   # calls arriving
cap_per_worker = 10

rev = 800
pen1 = 100   # 1-month delay
pen2 = 200   # 2-month delay
salary = 4000
hire_cost = 5000
fire_cost = 4000
W0 = 8

# --- Model --- #
m = LpProblem("Ghostbusters_Workforce_and_Scheduling", LpMaximize)

# Workforce variables (integers) #
W  = {t: LpVariable(f"W_{t}",  lowBound=0, cat=LpInteger) for t in months}
H  = {t: LpVariable(f"H_{t}",  lowBound=0, cat=LpInteger) for t in months}
F  = {t: LpVariable(f"F_{t}",  lowBound=0, cat=LpInteger) for t in months}

# Service variables by cohort and delay (continuous, calls can be fractional in LP; add Integer if needed) #
x_10 = LpVariable("x_1_0", lowBound=0)  # Jan calls served in Jan
x_11 = LpVariable("x_1_1", lowBound=0)  # Jan calls served in Feb
x_12 = LpVariable("x_1_2", lowBound=0)  # Jan calls served in Mar

x_20 = LpVariable("x_2_0", lowBound=0)  # Feb calls served in Feb
x_21 = LpVariable("x_2_1", lowBound=0)  # Feb calls served in Mar

x_30 = LpVariable("x_3_0", lowBound=0)  # Mar calls served in Mar

# Objective: revenue - penalties - salaries - hire/fire #
total_calls = x_10 + x_11 + x_12 + x_20 + x_21 + x_30
penalties   = pen1*(x_11 + x_21) + pen2*(x_12)

m += rev*total_calls - penalties \
     - salary*lpSum(W[t] for t in months) \
     - hire_cost*lpSum(H[t] for t in months) \
     - fire_cost*lpSum(F[t] for t in months)

# Cohort completion (all calls done by end of March) #
m += x_10 + x_11 + x_12 == D[1]   # January cohort
m += x_20 + x_21        == D[2]   # February cohort
m += x_30               == D[3]   # March cohort

# Monthly capacity constraints #
m += x_10                  <= cap_per_worker * W[1]                         # Jan
m += x_11 + x_20           <= cap_per_worker * W[2]                         # Feb
m += x_12 + x_21 + x_30    <= cap_per_worker * W[3]                         # Mar

# Workforce balance (hire/fire at the start of month) #
m += W[1] == W0 + H[1] - F[1]
m += W[2] == W[1] + H[2] - F[2]
m += W[3] == W[2] + H[3] - F[3]

# Solve #
m.solve()

# Report #
print("Status:", LpStatus[m.status])
print(f"Max Profit: ${value(m.objective):,.2f}\n")

print("Workforce plan (W=workers, H=hire, F=fire):")
for t in months:
    print(f" Month {t}: W={int(W[t].value())}, H={int(H[t].value())}, F={int(F[t].value())}")

print("\nService plan (calls served by cohort/delay):")
print(f" Jan cohort: on-time={x_10.value():.1f}, +1mo={x_11.value():.1f}, +2mo={x_12.value():.1f}")
print(f" Feb cohort: on-time={x_20.value():.1f}, +1mo={x_21.value():.1f}")
print(f" Mar cohort: on-time={x_30.value():.1f}")

# Sanity checks #
jan_cap = cap_per_worker * W[1].value()
feb_cap = cap_per_worker * W[2].value()
mar_cap = cap_per_worker * W[3].value()
print("\nCapacity check (used <= available):")
print(f" Jan used={x_10.value():.1f} / cap={jan_cap:.1f}")
print(f" Feb used={(x_11.value()+x_20.value()):.1f} / cap={feb_cap:.1f}")
print(f" Mar used={(x_12.value()+x_21.value()+x_30.value()):.1f} / cap={mar_cap:.1f}")


Status: Optimal
Max Profit: $150,000.00

Workforce plan (W=workers, H=hire, F=fire):
 Month 1: W=10, H=2, F=0
 Month 2: W=25, H=15, F=0
 Month 3: W=25, H=0, F=0

Service plan (calls served by cohort/delay):
 Jan cohort: on-time=100.0, +1mo=0.0, +2mo=0.0
 Feb cohort: on-time=250.0, +1mo=50.0
 Mar cohort: on-time=200.0

Capacity check (used <= available):
 Jan used=100.0 / cap=100.0
 Feb used=250.0 / cap=250.0
 Mar used=250.0 / cap=250.0
